In [1]:
!sudo apt-get install -qq -y timidity libsndfile1

In [2]:
# All the imports to deal with sound data
!pip -q install pydub numba==0.48 librosa music21

In [3]:
import tensorflow as tf
import tensorflow_hub as hub

import numpy as np
import matplotlib.pyplot as plt
import librosa
from librosa import display as librosadisplay

import logging
import math
import statistics
import sys

from IPython.display import Audio, Javascript
from scipy.io import wavfile

from base64 import b64decode

import music21
from pydub import AudioSegment

import warnings
warnings.filterwarnings('ignore')

logger = logging.getLogger()
logger.setLevel(logging.ERROR)

print("tensorflow: %s" % tf.__version__)
#print("librosa: %s" % librosa.__version__)

tensorflow: 2.6.0


In [ ]:
#@title Texto de título predeterminado
from lime import LIME
from spice import SPICE

config = {"model_load_path": "https://tfhub.dev/google/spice/2",
          "audio_path": "c-scale.wav",
          "num_segments": 100,
          "num_perturb": 150,
          "kernel_width": 0.25,
          "num_top_features": 4}

spice = SPICE(config)
spice.get_model()

lime = LIME(config, spice)

In [ ]:
#@title [Run this] Definition of the JS code to record audio straight from the browser

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=5):
  try:
    from google.colab import output
  except ImportError:
    print('No possible to import output from google.colab')
    return ''
  else:
    print('Recording')
    display(Javascript(RECORD))
    s = output.eval_js('record(%d)' % (sec*1000))
    fname = 'recorded_audio.wav'
    print('Saving to', fname)
    b = b64decode(s.split(',')[1])
    with open(fname, 'wb') as f:
      f.write(b)
    return fname

In [ ]:
#@title Select how to input your audio  { run: "auto" }
INPUT_SOURCE = 'https://storage.googleapis.com/download.tensorflow.org/data/c-scale-metronome.wav' #@param ["https://storage.googleapis.com/download.tensorflow.org/data/c-scale-metronome.wav", "RECORD", "UPLOAD", "./drive/My Drive/YOUR_MUSIC_FILE.wav"] {allow-input: true}

print('You selected', INPUT_SOURCE)

if INPUT_SOURCE == 'RECORD':
  uploaded_file_name = record(5)
elif INPUT_SOURCE == 'UPLOAD':
  try:
    from google.colab import files
  except ImportError:
    print("ImportError: files from google.colab seems to not be available")
  else:
    uploaded = files.upload()
    for fn in uploaded.keys():
      print('User uploaded file "{name}" with length {length} bytes'.format(
          name=fn, length=len(uploaded[fn])))
    uploaded_file_name = next(iter(uploaded))
    print('Uploaded file: ' + uploaded_file_name)
elif INPUT_SOURCE.startswith('./drive/'):
  try:
    from google.colab import drive
  except ImportError:
    print("ImportError: files from google.colab seems to not be available")
  else:
    drive.mount('/content/drive')
    # don't forget to change the name of the file you
    # will you here!
    gdrive_audio_file = 'YOUR_MUSIC_FILE.wav'
    uploaded_file_name = INPUT_SOURCE
elif INPUT_SOURCE.startswith('http'):
  !wget --no-check-certificate 'https://storage.googleapis.com/download.tensorflow.org/data/c-scale-metronome.wav' -O c-scale.wav
  uploaded_file_name = 'c-scale.wav'
else:
  print('Unrecognized input format!')
  print('Please select "RECORD", "UPLOAD", or specify a file hosted on Google Drive or a file from the web to download file to download')

In [ ]:
#@title LIME explanation of SPICE Pitch Detection { run: "auto" }
from utils import plot_stft, play_audio

NOTE = "C" #@param ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"] {allow-input: true}

config["audio_path"] = uploaded_file_name
audio_samples, sample_rate = spice.load_audio(config["audio_path"])

pitch_outputs, confidence_outputs, mean_outputs = spice.get_predictions(audio_samples)
predicted_notes = spice.get_notes(pitch_outputs, confidence_outputs)

X_top, X_bottom = lime.get_top_bottom(audio_samples, NOTE)

X_top_orig, X_bottom_orig = X_top.copy(), X_bottom.copy()

print("Predicted notes: ", predicted_notes)

print("ORIGINAL AUDIO")
# Original audio
plot_stft(audio_samples, sample_rate)
play_audio(audio_samples, sample_rate)

print("XTOP")
# Xtop
plot_stft(X_top["audio"], sample_rate)
play_audio(X_top["audio"], sample_rate)

print("XBOTTOM")
#Xbottom
plot_stft(X_bottom["audio"], sample_rate)
play_audio(X_bottom["audio"], sample_rate)

In [ ]:
#@title Do it again!

new_NOTE = "C" #@param ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"] {allow-input: true}
from_what = "Xtop" #@param ["Xtop", "Xbottom"] {allow-input: true}
allow_recursion = "No" #@param ["Yes", "No"] {allow-input: true}

if allow_recursion == "No": 
  if from_what == "Xtop":
    audio_samples = X_top_orig["audio"]
  elif from_what == "Xbottom":
    audio_samples = X_bottom_orig["audio"]

elif allow_recursion == "Yes":
  if from_what == "Xtop":
    audio_samples = X_top["audio"]
  elif from_what == "Xbottom":
    audio_samples = X_bottom["audio"]

pitch_outputs, confidence_outputs, mean_outputs = spice.get_predictions(audio_samples)
predicted_notes = spice.get_notes(pitch_outputs, confidence_outputs)

X_top, X_bottom = lime.get_top_bottom(audio_samples, NOTE)

print("Predicted notes: ", predicted_notes)

print("ORIGINAL AUDIO")
# Original audio
plot_stft(audio_samples, sample_rate)
play_audio(audio_samples, sample_rate)

print("XTOP")
# Xtop
plot_stft(X_top["audio"], sample_rate)
play_audio(X_top["audio"], sample_rate)

print("XBOTTOM")
#Xbottom
plot_stft(X_bottom["audio"], sample_rate)
play_audio(X_bottom["audio"], sample_rate)